In [1]:

import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import io
import itertools
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout

In [2]:
batch_size = 32
img_height = 128
img_width = 128

In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  'data2/train',
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')

Found 7207 files belonging to 8 classes.


In [4]:
val_ds =  tf.keras.utils.image_dataset_from_directory(
  'data/test',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')

Found 2425 files belonging to 8 classes.


In [5]:
class_names = train_ds.class_names
print(class_names)

['Forward', 'Next', 'Pause', 'Play', 'Previous', 'Reverse', 'Volume Down', 'Volume Up']


In [6]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 128, 128, 1)
(32,)


In [7]:
for images, labels in val_ds:
    images_val = images.numpy()
    labels_val = labels.numpy()


In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [9]:
normalization_layer = layers.Rescaling(1./255)

In [10]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

0.0 1.0


In [11]:
for images, labels in val_ds:
    images_val = images.numpy()
    labels_val = labels.numpy()

In [12]:
log_dir = "Logs\\fit\\" + "run-1"

In [13]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
    cm (array, shape = [n, n]): a confusion matrix of integer classes
    class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(12, 12))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)

    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
    return figure

In [14]:
def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside the notebook.
    plt.close(figure)
    
    buf.seek(0)
    
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

In [15]:
# Define a file writer variable for logging purposes
file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(images_val)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
    
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=['Forward', 'Next', 'Pause', 'Play', 'Previous', 'Reverse', 'Volume Down', 'Volume Up'])
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [16]:
# Defining the callbacks
cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [17]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 1)),
  layers.Conv2D(32, 5, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(units=128, activation='relu'),
  layers.Dropout(0.40),
  layers.Dense(units=64, activation='relu'),
  layers.Dropout(0.40),
  layers.Dense(num_classes)
])

In [18]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 128, 128, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        9

In [20]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    mode = 'auto',
    min_delta = 0,
    patience = 5,
    verbose = 0, 
    restore_best_weights = True
)

In [25]:
epochs=20
model.fit(
  train_ds,
  validation_data=val_ds,
  callbacks = [tensorboard_callback, cm_callback,early_stopping],
  epochs=epochs
)

Epoch 1/20
226/226 [==============================] - 106s 468ms/step - loss: 0.0147 - accuracy: 0.9954 - val_loss: 1.2471 - val_accuracy: 0.6882
Epoch 2/20
226/226 [==============================] - 105s 466ms/step - loss: 0.0111 - accuracy: 0.9961 - val_loss: 0.8634 - val_accuracy: 0.7014
Epoch 3/20
226/226 [==============================] - 105s 466ms/step - loss: 0.0080 - accuracy: 0.9982 - val_loss: 1.1800 - val_accuracy: 0.5674
Epoch 4/20
226/226 [==============================] - 105s 466ms/step - loss: 0.0088 - accuracy: 0.9969 - val_loss: 0.4289 - val_accuracy: 0.8722
Epoch 5/20
226/226 [==============================] - 106s 468ms/step - loss: 0.0175 - accuracy: 0.9951 - val_loss: 0.8480 - val_accuracy: 0.7390
Epoch 6/20
226/226 [==============================] - 105s 466ms/step - loss: 0.0096 - accuracy: 0.9971 - val_loss: 0.2738 - val_accuracy: 0.9254
Epoch 7/20
226/226 [==============================] - 107s 472ms/step - loss: 0.0103 - accuracy: 0.9976 - val_loss: 0.6873 -

In [253]:
model_json = model.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
model.save_weights('model-bw.h5')
print('Weights saved')


Model Saved
Weights saved


In [38]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  '../data2/train',
  image_size=(img_height, img_width),
  batch_size=batch_size,
  color_mode='grayscale')

Found 1352 files belonging to 8 classes.


In [39]:
class_names = test_ds.class_names
print(class_names)

['Forward', 'Next', 'Pause', 'Play', 'Previous', 'Reverse', 'Volume Down', 'Volume Up']


In [40]:
model.evaluate(test_ds)

43/43 [==============================] - 6s 141ms/step - loss: 0.6360 - accuracy: 0.9194


[0.6359869837760925, 0.9193786978721619]

In [1]:
# Loading the Tensorboard extension
%load_ext tensorboard
%tensorboard --logdir "Logs\fit\run-1"

In [252]:
import cv2
import numpy as np
img = tf.keras.utils.load_img(
    "data2/test/Forward/2.jpg", target_size=(img_height, img_width,),grayscale=True
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])


x="This image most likely belongs to {} with a {:.2f} percent confidence.".format(class_names[np.argmax(score)], 100 * np.max(score))
print(x)
print(np.argmax(score))


This image most likely belongs to Forward with a 100.00 percent confidence.
0


/Users/anishasaxena/opt/anaconda3/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


32512